In [15]:
import pandas as pd 
from helpers.preprocessing import *
import os
from ast import literal_eval


In [12]:
rosmap_original = pd.read_csv("../data/raw_data_frequencies/ROSMAP_annotation_full_metadata.csv")

In [45]:
rosmap_original["compound_name_preprocessed"] = rosmap_original["compound_name"].apply(lambda x: (preprocess_chemical([x]))[0])
# all_data["synonyms"] = all_data["synonyms"].apply(tuple)
# all_data.drop_duplicates(subset=["synonyms", "dataset"], inplace=True)
# all_data.drop(all_data[all_data["synonyms"].apply(len) == 0].index, inplace=True)

rosmap_original_collapsed = rosmap_original.groupby(["compound_name_preprocessed"])["featureID"].apply(lambda x: list(set(x))).reset_index()
rosmap_original_collapsed.set_index("compound_name_preprocessed", inplace=True)
rosmap_original_collapsed_dict = rosmap_original_collapsed.to_dict()["featureID"]
rosmap_original["nonzero_DF"] = rosmap_original["DF"].apply(lambda x : str(x) if x != 0 else "ZERO")

rosmap_df_dict = dict(zip(rosmap_original["nonzero_DF"][rosmap_original["nonzero_DF"] != "ZERO"].to_list(), rosmap_original["featureID"][rosmap_original["nonzero_DF"] != "ZERO"].to_list()))

In [58]:
all_data = pd.read_csv("../data/output/all_additional_datasets.csv")
rosmap_data = all_data[all_data['dataset'] == 'rosmap']

In [59]:
def map_synonym(synonym_list, dictionary):
    for synonym in synonym_list:
        try:
            return dictionary[synonym]
        except KeyError:
            continue
    return None    
rosmap_data["matched_features"] = rosmap_data["synonyms"].apply(lambda x : map_synonym(literal_eval(x), rosmap_original_collapsed_dict))
rosmap_data["nonzero_df"] = rosmap_data["DF"].apply(lambda x: str(x) if x != 0 else "ZERO")

rosmap_data["matched_features"] = rosmap_data.apply(lambda x: [rosmap_df_dict[x["nonzero_df"]]] if (x["matched_features"] is None and x["nonzero_df"] != "ZERO") else x["matched_features"], axis=1)


/tmp/ipykernel_8872/1503647142.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rosmap_data["matched_features"] = rosmap_data["synonyms"].apply(lambda x : map_synonym(literal_eval(x), rosmap_original_collapsed_dict))
/tmp/ipykernel_8872/1503647142.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rosmap_data["nonzero_df"] = rosmap_data["DF"].apply(lambda x: str(x) if x != 0 else "ZERO")
/tmp/ipykernel_8872/1503647142.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [62]:
rosmap_data

,X.Scan.,synonyms,text,name_used,site,chemsource_output_gpt-4o,chemsource_output_gpt-4o_classification,chemsource_output_gpt-4o_classprobs,DF,dataset,synonyms_lower,matched_features,nonzero_df
2455,NaN,"('Histidine', 'H-his-oh', 'Glyoxaline-5-alanin...",Histidine (symbol His or H) is an essential am...,Histidine,WIKIPEDIA,"('FOOD', [('FO', -0.006872610189020634), ('OD'...",FOOD,{'FOOD': 1.0},0.919926,rosmap,"['histidine', 'h-his-oh', 'glyoxaline-5-alanin...",[1759],0.919925512104283
2456,NaN,"('N-acetyl-l-methionine', 'N-acetylmethionine'...",Two experiments were conducted to investigate...,N-acetyl-l-methionine,PUBMED,"('INFO', [('INFO', -0.1612294614315033)])",INFO,{'INFO': 0.851096754819522},0.951583,rosmap,"['n-acetyl-l-methionine', 'n-acetylmethionine'...",[7404],0.951582867783985
2457,NaN,"('Lactulose', 'Bifiteral', 'Cephulac', 'Chronu...",Lactulose is a non-absorbable sugar used in th...,Lactulose,WIKIPEDIA,"('MEDICAL, FOOD', [('MED', -2.4391956685576588...","MEDICAL, FOOD","{'MEDICAL': 0.9999756083407958, 'FOOD': 0.9999...",0.500931,rosmap,"['lactulose', 'bifiteral', 'cephulac', 'chronu...",[2586],0.500931098696462
2458,NaN,"('Phenylacetylglutamine', 'Phenylacetyl l-glut...",Phenylacetylglutamine is a product formed by t...,Phenylacetylglutamine,WIKIPEDIA,"('ENDOGENOUS', [('END', -1.1472419600977446e-0...",ENDOGENOUS,{'ENDOGENOUS': 1.0},0.361266,rosmap,"['phenylacetylglutamine', 'phenylacetyl l-glut...",[7600],0.361266294227188
2459,NaN,"('Carbamazepine', 'Tegretol', 'Carbamazepen', ...","Carbamazepine, sold under the brand name Tegre...",Carbamazepine,WIKIPEDIA,"('MEDICAL', [('MED', 0.0), ('ICAL', 0.0)])",MEDICAL,{'MEDICAL': 1.0},0.003724,rosmap,"['carbamazepine', 'tegretol', 'carbamazepen', ...",[15683],0.0037243947858473
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2567,NaN,"('Coronaric acid', 'Leukotoxin a (9,10-eode)',...",Coronaric acid (leukotoxin or leukotoxin A) is...,Coronaric acid,WIKIPEDIA,"('ENDOGENOUS, FOOD', [('END', -0.0054735173471...","ENDOGENOUS, FOOD","{'ENDOGENOUS': 0.9933776157671295, 'FOOD': 0.9...",0.000000,rosmap,"['coronaric acid', 'leukotoxin a (9,10-eode)',...",None,ZERO
2568,NaN,"('Ephedrine', 'Ephedrin', 'Lephedrine', 'Lexof...",Ephedrine is a central nervous system (CNS) st...,Ephedrine,WIKIPEDIA,"('MEDICAL, FOOD', [('MED', -1.6240566083070007...","MEDICAL, FOOD","{'MEDICAL': 0.9999983759447105, 'FOOD': 0.9999...",0.005587,rosmap,"['ephedrine', 'ephedrin', 'lephedrine', 'lexof...",[15304],0.0055865921787709
2569,NaN,"('Dodecanedioic acid', 'Decamethylenedicarboxy...",Dodecanedioic acid (DDDA) is a dicarboxylic ac...,Dodecanedioic acid,WIKIPEDIA,"('INDUSTRIAL, MEDICAL, PERSONAL CARE', [('IND'...","INDUSTRIAL, MEDICAL, PERSONAL CARE","{'INDUSTRIAL': 0.9858108394911275, 'MEDICAL': ...",0.188082,rosmap,"['dodecanedioic acid', 'decamethylenedicarboxy...",[10749],0.188081936685289
2570,NaN,"('Lysyl-isoleucine', 'Lys-ile', 'H-lys-ile-oh'...",Because of their ability to accumulate at sit...,Lys-ile,PUBMED,"('INFO', [('INFO', -4.429896944202483e-05)])",INFO,{'INFO': 0.9999557020117429},0.124767,rosmap,"['lysyl-isoleucine', 'lys-ile', 'h-lys-ile-oh'...",[9514],0.124767225325885


In [57]:
rosmap_data.drop(columns=["nonzero_df"], inplace=True)
rosmap_data.to_csv("../data/output/rosmap_data_with_features.csv", index=False)

/tmp/ipykernel_8872/2329077580.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rosmap_data.drop(columns=["nonzero_df"], inplace=True)


In [39]:
rosmap_original_collapsed

,featureID
compound_name_preprocessed,
"""1,6-anhydro-b-glucose""",[6768]
"""3,4,5-trimethoxyphenol ""","[7382, 1286]"
"""n,n,n-trimethyllysine""",[14736]
(1r)-nopol,[6419]
"(2r,3r,4s,5s,6r)-2-[1,7-bis(4-hydroxyphenyl)heptan-3-yloxy]-6-[[(2s,3r,4r)-3,4-dihydroxy-4-(hydroxymethyl)oxolan-2-yl]oxymethyl]oxane-3,4,5-triol",[10035]
...,...
Val-phe,"[11300, 7007]"
Val-val,[8561]
Valine,[1062]
